# Running ELM on NERSC
## Navigating NERSC
### Open NERSC user account (for new users)
Follow the  <a href="https://docs.nersc.gov/accounts/">instructions</a> to open an account on NERSC: 
### Connecting to NERSC
Enable <a href="https://docs.nersc.gov/connect/mfa/">Multi Factor Authetication (MFA)</a> to access NERSC resources.

`ssh <user>@perlmutter-p1.nersc.gov` (for Perlmutter)<br>
`ssh <user>@cori.nersc.gov` (for Cori)

### Data transfer between NERSC and local machine
Transferring <a href="https://docs.nersc.gov/services/scp/">a file using scp and directory using tar+ssh</a> into/out of NERSC. 
For transferring large files/folders they should be zipped before transferring.<br>

## Setting up <a href="https://elm-unofficial-guide.readthedocs.io/en/latest/getting_started.html">ELM</a>
This documentation is generated using <a href="https://elm-unofficial-guide.readthedocs.io/en/latest/index.html">ELM</a>, <a href="https://escomp.github.io/ctsm-docs/versions/master/html/index.html">CTSM</a>, <a href="https://escomp.github.io/CESM/release-cesm2/index.html#">CESM</a> and <a href="https://escomp.github.io/CESM/release-cesm2/index.html">CIME</a> documentation and an <a href=https://github.com/donghuix/Setup-E3SM-Mac/blob/master/Setup-E3SM-on-Mac.md>example</a>.<br>
### Get the code 
Generate an <a href="https://docs.github.com/en/authentication/connecting-to-github-with-ssh/generating-a-new-ssh-key-and-adding-it-to-the-ssh-agent">SSH key on GitHub</a>  before cloning.<br>
`git clone https://github.com/E3SM-Project/E3SM`<br>
`cd E3SM`<br>
`git submodule update --init --recursive`<br>

### Create user defined domain and surface daatset
A <a href=https://github.com/ftzahura/SettingUpELM/blob/main/CreateSurfdataDomainNetcdf.ipynb>python</a> script was generated to create unstructured domain and surface dataset for user define watershed.<br>
The python script was generated following this <a href="https://github.com/bishtgautam/matlab-script-for-clm-sparse-grid">Matlab script </a> to generate similar outputs.

Transfer the files on NERSC if using local machine<br>
`scp /<local-directory>/<name-of-domain-file> user@dtn01.nersc.gov:<dir-path-to-domain-file>`<br>
`scp /<local-directory>/<name-of-surface-file> user@dtn01.nersc.gov:<dir-path-to-surface-file>`

### Create a new case 
Specify resolution `-res <>` and compset `-compset <>` on an E3SM-supported machine.<br>
`cd cime/scripts`<br>
`./create_newcase -case <casename> -res ELM_USRDAT -compset IELM -project <yourprojectallocation>` <br>

Go to the case directory<br>
`cd <casename>`<br>

### Making modifications to the case and case setup

#### Specify run period
Change the default 5-day run to be 1-year run and set up the start date<br>
`./xmlchange STOP_N=1,STOP_OPTION=nyears`<br>
`./xmlchange RUN_STARTDATE=1980-01-01`
#### Location of outputs
Query the dir where the output will be saved<br>
`./xmlquery RUNDIR`
#### Wallclock time and number of processors
Set the wallclock time anf number of processors for simualtion. Higher wallclock time and number of processors would cause longer wait period. Default `NTASKS` is 256 (4 nodes) <br> 
`./xmlchange JOB_WALLCLOCK_TIME="02:00:00" --subgroup case.run`<br>
`./xmlchange NTASKS=<number-of-processors>`<br>
`./xmlquery JOB_QUEUE,NTASKS,JOB_WALLCLOCK_TIME`<br>

#### Point to the new domain file:<br>
`./xmlchange LND_DOMAIN_FILE=<name-of-domain-file>`<br>
`./xmlchange ATM_DOMAIN_FILE=<name-of-domain-file>`<br>
`./xmlchange LND_DOMAIN_PATH=<dir-path-to-domain-file>`<br>
`./xmlchange ATM_DOMAIN_PATH=<dir-path-to-domain-file>`<br>

#### Point to the new surface dataset:<br>
`cat >> user_nl_elm << EOF`<br>
`fsurdat = '<full-path-and-filename-of-new-surface-dataset>'`<br>
`EOF`<br>

#### Customizing `user_nl_elm`
The defaulut output from ELM is monthy history files with >200 variables. To only get the variables of interest the `user_nl_elm` needs to be edited. `hist_empty_htapes` removes all default variables and `hist_fincl1` only includes select variables to the monthly history files.<br>
`cat >> user_nl_elm << EOF`<br>
`&elm_inparm`<br>
  `hist_empty_htapes = .true.`<br>
  `hist_fincl1 = 'TG', 'TV', 'FSA'`<br>
`/`<br>
`EOF`<br>
Examples of customizing `user_nl_elm` can be found <a href="https://escomp.github.io/ctsm-docs/versions/master/html/users_guide/setting-up-and-running-a-case/customizing-the-clm-namelist.html">in the CLM documnetation</a>. 
The output frequency is monthly averged by default. the default averaging output can be changed by modiying `user_nl_elm`:
1. Using `hist_avgflag_pertape` to give a default value for each history stream
2. Using hist_fincl*, while addind fields, the averaging flag van be assigned after the field name with a colon (:) (for example ‘TSOI:X’, would output the maximum of TSOI).<br>
The types of averaging that can be done are:
- A Average, over the output interval.
- I Instantaneous, output the value at the output interval.
- X Maximum, over the output interval.
- M Minimum, over the output interval.
3. The default monthly can be changed by assigning `hist_nhtfrq` to -1 (hourly),-6 (six hourly),-24 (daily), 0 (monthly), 1 (each model time step)


#### Changing DATM forcing
The default `DATM_MODE` in ELM is `CLM_QIAN`. To use NLDAS forcing change the forcing mode and period.<br>
`./xmlchange DATM_MODE=CLMNLDAS2`<br>
`./xmlchange DATM_CLMNCEP_YR_START=1980`<br>
`./xmlchange DATM_CLMNCEP_YR_END=2018`<br>
`./xmlchange DATM_CLMNCEP_YR_ALIGN=1980`<br>
`./xmlquery  DATM_MODE,DATM_CLMNCEP_YR_START,DATM_CLMNCEP_YR_END,DATM_CLMNCEP_YR_ALIGN`

#### Case setup 
`./case.setup`

#### Update datm.streams files
`CLMNLDAS2` points to folder `/global/cfs/cdirs/e3sm/inputdata/atm/datm7/atm_forcing.datm7.NLDAS2.0.125d.v1`. The `atm_forcing.datm7.NLDAS2.0.125d.v1` folder has subfolders for Precip, Solar, and TPQW. After case setup, in Casedocs folder three data stream files are generated: datm.streams.txt.CLMNLDAS2.Precip, datm.streams.txt.CLMNLDAS2.Solar, datm.streams.txt.CLMNLDAS2.TPQW. 

case submission was aborted bacause of file missing error: `/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.clm/domain.lnd.0.125nldas2_0.125nldas2.190410.nc`. In `/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.clm/` folder, file `domain.lnd.0.125nldas2_0.125nldas2.190410.nc` did not exist. Downloaded the file (<a href=https://svn-ccsm-inputdata.cgd.ucar.edu/trunk/inputdata/share/domains/domain.clm/>link</a>) and changed the data stream files to point to the file location:

`cp ${CASE_DIR}/${CASE_NAM}/CaseDocs/datm.streams.txt.CLMNLDAS2.Precip ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.Precip`<br>
`cp ${CASE_DIR}/${CASE_NAM}/CaseDocs/datm.streams.txt.CLMNLDAS2.Solar ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.Solar`<br>
`cp ${CASE_DIR}/${CASE_NAM}/CaseDocs/datm.streams.txt.CLMNLDAS2.TPQW ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.TPQW`<br>

`chmod +rw ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.Precip`<br>
`chmod +rw ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.Solar`<br>
`chmod +rw ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.TPQW`<br>

`sed -i "s@/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.clm@<file-path>@g" ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.Precip`<br>
`sed -i "s@/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.clm@<file-path>@g" ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.Solar`<br>
`sed -i "s@/global/cfs/cdirs/e3sm/inputdata/share/domains/domain.clm@<file-path>@g" ${CASE_DIR}/${CASE_NAM}/user_datm.streams.txt.CLMNLDAS2.TPQW`<br>


### Build and Submit the case and get email updates
As changes were made after `./case.setup`, it needs to be reset.<br>
`./case.setup --reset`<br>
<a href=https://esmci.github.io/cime/versions/master/html/users_guide/building-a-case.html>Build the case</a>, submit and get email updates.<br>
`./case.build`<br>
`./case.submit --mail-user <user-email> --mail-type all` <br>
See job status<br>
`squeue -u <username>`<br>
`squeue --start -j <JOBID>`


### <a href=https://escomp.github.io/ctsm-docs/versions/release-clm5.0/html/users_guide/running-special-cases/Spinning-up-the-Satellite-Phenology-Model-CLMSP-spinup.html>Spin up ELM SP (satellite phenology)</a>
Spinning up ELM-SP is straightforward. To spin up, 50 years or more (user's choice) need to be run and the final restart file for initial conditions is used in other simulations. For 50 year spin up, loop over 1 year atmospheric forcing 50 times or 5 years 10 times, etc. Set `RESUBMIT` to specify the number of times to loop over the atmospheric forcing. As `RESUBMIT=0` loops one time, to loop 10 times set it to 9.<br>
`./xmlchange STOP_N=5,STOP_OPTION=nyears  # year period per run/resubmit`<br>
`./xmlchange RESUBMIT=9` <br>
After spin up is finished, create a new case and in `user_nl_elm` point to the final restart file from the spin up simulation, which is located in the `RUNDIR` folder:<br>
`cat >> user_nl_elm << EOF`                     
`finidat = '<full-path-and-name-of-restart-file>'`